In [1]:
#importing library
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import os
import glob

In [2]:
##is the problem scaled??

scale=True

In [3]:
#defining image saving function

def save_fig(_id, tight_layout=True, resolution=300):
    path = os.path.join("./output/quantile_figs/just_sim", _id + ".png")
    print("Saving figure", _id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=resolution)

In [4]:
#loading user data and getting number of stages

user_data=pd.read_csv('input/user_data.csv')
n_stages=user_data.iloc[0,1]
user_data

variables  values
0                                   Number of stages      21
1                          month for initial storage       3
2                            day for initial storage      31
3  nuber of rows from begening of USACE forecsat ...       3
4   nuber of rows from End of USACE forecsat to drop       0
5                              number of small trees       7

In [5]:
params=pd.read_excel('input/Cascade_info.xlsx',sheet_name='plotting_info')
name=params['name']
name=sorted(name)
k=params['k']
min_release=params['release_min']
max_powerhouse=params['max_powerhouse']
max_spill=params['max_spill']
baseline=params['baseline']
flood_control=params['flood_control']
params=params.sort_values(by='name',ascending=True)
params=params.reset_index(drop=True)
params

name      k  max_powerhouse  max_spill  baseline  flood_control  \
0   BB   4.60          103000     270000   1810000        1749000   
1   FP   5.84           31721     230000  18463000       17492000   
2   FR   8.50           37647     508000   5293000        4307000   
3   GA  12.30           41000     660000  23451000       21956000   
4   GP   3.45           36000     345000    428000         374000   
5   OH  12.45           54600      80000  22983000       21876000   

   release_min  release_max  
0            0       103000  
1            6        39200  
2            9        44496  
3            9        41000  
4            9        36000  
5            0        54000

In [6]:
name

['BB', 'FP', 'FR', 'GA', 'GP', 'OH']

In [7]:
dir_list=os.listdir("output\quantile_files\main")
dir_list

['BB_release.xlsx',
 'BB_spill.xlsx',
 'BB_storage.xlsx',
 'BB_total_pflow.xlsx',
 'FP_release.xlsx',
 'FP_spill.xlsx',
 'FP_storage.xlsx',
 'FP_total_pflow.xlsx',
 'FR_release.xlsx',
 'FR_spill.xlsx',
 'FR_storage.xlsx',
 'FR_total_pflow.xlsx',
 'GA_release.xlsx',
 'GA_spill.xlsx',
 'GA_storage.xlsx',
 'GA_total_pflow.xlsx',
 'GP_release.xlsx',
 'GP_spill.xlsx',
 'GP_storage.xlsx',
 'GP_total_pflow.xlsx',
 'OH_release.xlsx',
 'OH_spill.xlsx',
 'OH_storage.xlsx',
 'OH_total_pflow.xlsx']

In [8]:
release,power,spill,storage=[],[],[],[]
for file in dir_list:
    temp=pd.read_excel(os.path.join("output\quantile_files\main",file))
    if file.endswith('release.xlsx'):
        release.append(temp)
    elif file.endswith('spill.xlsx'):
        spill.append(temp)
    elif file.endswith('storage.xlsx'):
        storage.append(temp)
    elif file.endswith('pflow.xlsx'):
        power.append(temp)

In [9]:
release[0]

x1         x2         x3         x4         x5         x6         x7  \
0  0.00  78.941147  50.205275  37.897183  12.594689   1.543210   1.543210   
1  0.10  78.941147  50.205275  37.897183  54.600000  54.600000  45.477586   
2  0.25  78.941147  50.205275  43.053229  54.600000  54.600000  54.600000   
3  0.50  78.941147  50.205275  43.053229  55.200000  54.600000  54.600000   
4  0.75  78.941147  53.322139  47.028735  55.200000  54.600000  54.600000   
5  0.90  78.941147  53.322139  47.028735  65.541418  58.961986  55.999414   
6  1.00  78.941147  53.322139  47.028735  71.342817  58.961986  65.680000   

          x8         x9        x10  ...        x13        x14        x15  \
0   9.214269  18.569172   6.484055  ...   1.543210  15.970465  37.618335   
1  54.600000  52.237831   9.895992  ...  37.731394  34.168782  51.619785   
2  54.600000  52.237831  15.458924  ...  37.731394  34.168782  51.648428   
3  54.660000  54.600000  15.458924  ...  50.850726  48.965291  55.400000   
4  54.680000  54.600000  54.700000  ...  53.994709  49.673064  57.000000   
5  55.222000  54.600000  55.120000  ...  54.600000  52.208795  57.382484   
6  58.240000  54.920000  73.580376  ...  58.480000  54.600000  69.092593   

         x16        x17         x18        x19        x20         x21  \
0  17.713597   1.543210   15.092978  16.218373   1.543210    1.543210   
1  47.665352  12.154238   50.823045  34.656529  21.594629   17.987564   
2  54.600000  29.715269   54.600000  54.600000  34.783225   28.183667   
3  55.840000  29.715269   54.600000  54.600000  49.551552   41.621596   
4  55.840000  33.667699   54.940000  54.600000  49.825736   46.074417   
5  57.580215  40.746565   77.673793  58.140975  63.528807   51.045629   
6  62.524097  57.984438  101.646091  88.178567  81.972031  100.584119   

         x22  
0   1.543210  
1  14.451617  
2  25.909340  
3  35.169823  
4  42.580904  
5  53.884172  
6  91.773235  

[7 rows x 22 columns]

In [10]:
# importing Root  node solutions of shifted tree

shifted_solutions=pd.read_csv('output/node_solutions/shifted_tree/shifted_Root_node_solutions.csv',\
                              index_col=0)
shifted_solutions.head(8)

BB_out     FP_out     FR_out  GA_out  GP_out     OH_out
release_0  78.941147  15.300000  37.647059    25.1    36.0  51.822201
release_1  62.924129  15.300000  37.647059    25.1    36.0  36.385182
release_2  64.282454  15.300000  37.647059    25.1    36.0  37.743508
release_3  77.476973  15.300000  37.647059    25.1    36.0  50.938027
release_4  77.006172  15.300000  37.647059    25.1    36.0  50.467226
release_5  75.747027  12.558451  37.647059    25.1    36.0  54.600000
release_6  83.717399  15.300000  37.647059    25.1    36.0  57.178453
release_7  78.592359  15.300000  37.647059    25.1    36.0  52.053412

In [11]:
#defining release plot function

def plot_release(a,b,shifted,scale=scale,n=n_stages,):
    if scale:
        constant=1e3
    else:
        constant=1
    print(f'constant={constant}')
    
    ##defining quantiles
    q_0=a.iloc[0][1:].to_numpy()*constant
    q_10=a.iloc[1][1:].to_numpy()*constant
    q_25=a.iloc[2][1:].to_numpy()*constant
    q_50=a.iloc[3][1:].to_numpy()*constant
    q_75=a.iloc[4][1:].to_numpy()*constant
    q_90=a.iloc[5][1:].to_numpy()*constant
    q_100=a.iloc[6][1:].to_numpy()*constant
    
    ##defining forecast data
    lower=np.full(shape=n,fill_value=b['release_min'],)
    
    ##x for main tree
    x=np.arange(0,n)
    
    ##python starts indexes from 0 lets add 1 to it for beter visualisation
    x=x+1
    
    ## xx for shifted tree
    xx=np.arange(0,len(shifted))
    xx=xx+1
    
    ##plot the figure
    fig=plt.figure(figsize=(40,20),dpi=300)
    ax=fig.add_subplot(111,label='1')
    c='#33d7ff'
    ax.plot(x,q_50,color='#3337ff',linewidth=5,label='Median')
    ax.plot(x,lower,color='r',linewidth=5,label='Lower bound')
    ax.plot(xx,shifted*constant,'g:',linewidth=5,label="roling horizon")
    ax.fill_between(x,q_0,q_10,alpha=.1,color=c)
    ax.fill_between(x,q_10,q_25,alpha=.3,color=c)
    ax.fill_between(x,q_25,q_50,alpha=.5,color=c)
    ax.fill_between(x,q_50,q_75,alpha=.3,color=c)
    ax.fill_between(x,q_75,q_90,alpha=.2,color=c)
    ax.fill_between(x,q_90,q_100,alpha=.1,color=c)
    title=b['name']
    plt.title(title,fontsize=30)
    plt.xlabel(f'Stage ({n} day horizon)',fontsize=30)
    plt.ylabel('Release (cfs)',fontsize=30)
    plt.grid(True,which='major',axis='both')
    plt.xticks(ticks=x)
    ax.tick_params(axis='both', which='major', labelsize=30)
    plt.xlim([1,n])
    ax.legend(fontsize=30)
    save_fig(f'release for {title}')
    plt.close('all')

In [12]:
#plotting for release
kk=0
for dam,i in zip(release,range(len(name))):
    p=params.iloc[i,:]
    plot_release(dam,p,shifted_solutions.iloc[0:8,kk])
    kk+=1

constant=1000.0
Saving figure release for BB
constant=1000.0
Saving figure release for FP
constant=1000.0
Saving figure release for FR
constant=1000.0
Saving figure release for GA
constant=1000.0
Saving figure release for GP
constant=1000.0
Saving figure release for OH


In [13]:
plt.close('all')

In [14]:
#defining power plot function
def plot_power(a,b,shifted,scale=scale,n=n_stages,):
    if scale:
        constant=1e3
    else:
        constant=1
    print(constant)
    q_0=a.iloc[0][1:].to_numpy()*constant
    q_10=a.iloc[1][1:].to_numpy()*constant
    q_25=a.iloc[2][1:].to_numpy()*constant
    q_50=a.iloc[3][1:].to_numpy()*constant
    q_75=a.iloc[4][1:].to_numpy()*constant
    q_90=a.iloc[5][1:].to_numpy()*constant
    q_100=a.iloc[6][1:].to_numpy()*constant
    q_100
    upper=np.full(shape=n,fill_value=b['max_powerhouse'],)
    lower=np.full(shape=n,fill_value=b['release_min']*b['k'],)
    x=np.arange(0,n)
    x=x+1
    xx=np.arange(0,len(shifted))
    xx=xx+1
    fig=plt.figure(figsize=(40,20),dpi=300)
    ax=fig.add_subplot(111,label='1')
    c='#33d7ff'
    ax.plot(x,q_50,color='#3337ff',linewidth=5,label='Median')
    ax.plot(x,upper,color='k',linewidth=5,label='Upper bound')
    ax.plot(x,lower,color='r',linewidth=5,label='Lower bound')
    ax.plot(xx,shifted*constant,'g:',linewidth=5,label="roling horizon")
    ax.fill_between(x,q_0,q_10,alpha=.1,color=c)
    ax.fill_between(x,q_10,q_25,alpha=.3,color=c)
    ax.fill_between(x,q_25,q_50,alpha=.5,color=c)
    ax.fill_between(x,q_50,q_75,alpha=.3,color=c)
    ax.fill_between(x,q_75,q_90,alpha=.2,color=c)
    ax.fill_between(x,q_90,q_100,alpha=.1,color=c)
    title=b['name']
    plt.title(title,fontsize=30)
    plt.xlabel(f'Stage ({n} day horizon)',fontsize=30)
    plt.ylabel('Powerhouse (cfs)',fontsize=30)
    plt.grid(True,which='major',axis='both')
    plt.xticks(ticks=x)
    ax.tick_params(axis='both', which='major', labelsize=30)
    plt.xlim([1,n])
    ax.legend(fontsize=30)
    save_fig(f'Power for {title}')
    plt.close('all')

In [15]:
#plotting for power
kk=0
for dam,i in zip(power,range(len(name))):
    p=params.iloc[i,:]
    plot_power(dam,p,shifted_solutions.iloc[24:,kk])
    kk+=1

1000.0
Saving figure Power for BB
1000.0
Saving figure Power for FP
1000.0
Saving figure Power for FR
1000.0
Saving figure Power for GA
1000.0
Saving figure Power for GP
1000.0
Saving figure Power for OH


In [16]:
plt.close('all')

In [17]:
#defining spill plot function
def plot_spill(a,b,shifted,scale=scale,n=n_stages):
    if scale:
        constant=1e3
    else:
        constant=1
    print(constant)
    q_0=a.iloc[0][1:].to_numpy()*constant
    q_10=a.iloc[1][1:].to_numpy()*constant
    q_25=a.iloc[2][1:].to_numpy()*constant
    q_50=a.iloc[3][1:].to_numpy()*constant
    q_75=a.iloc[4][1:].to_numpy()*constant
    q_90=a.iloc[5][1:].to_numpy()*constant
    q_100=a.iloc[6][1:].to_numpy()*constant
    upper=np.full(shape=n,fill_value=b["max_spill"],)
    x=np.arange(0,n)
    x=x+1
    xx=np.arange(0,len(shifted))
    xx=xx+1
    fig=plt.figure(figsize=(40,20),dpi=300)
    ax=fig.add_subplot(111,label='1')
    c='#33d7ff'
    ax.plot(x,q_50,color='#3337ff',linewidth=5,label='Median')
    ax.plot(x,upper,color='k',linewidth=5,label='Upper bound')
    ax.plot(xx,shifted*constant,'g:',linewidth=5,label="roling horizon")
    ax.fill_between(x,q_0,q_10,alpha=.1,color=c)
    ax.fill_between(x,q_10,q_25,alpha=.3,color=c)
    ax.fill_between(x,q_25,q_50,alpha=.5,color=c)
    ax.fill_between(x,q_50,q_75,alpha=.3,color=c)
    ax.fill_between(x,q_75,q_90,alpha=.2,color=c)
    ax.fill_between(x,q_90,q_100,alpha=.1,color=c)
    title=b['name']
    plt.title(title,fontsize=30)
    plt.xlabel(f'Stage ({n} day horizon)',fontsize=30)
    plt.ylabel('Spill (cfs)',fontsize=30)
    plt.grid(True,which='major',axis='both')
    plt.xticks(ticks=x)
    ax.tick_params(axis='both', which='major', labelsize=30)
    plt.xlim([1,n])
    ax.legend(fontsize=30)
    save_fig(f'spill for {title}')
    plt.close('all')    

In [18]:
#plotting for spill
kk=0
for dam,i in zip(spill,range(len(name))):
    p=params.iloc[i,:]
    plot_spill(dam,p,shifted_solutions.iloc[8:9+7,kk])
    kk+=1

1000.0
Saving figure spill for BB
1000.0
Saving figure spill for FP
1000.0
Saving figure spill for FR
1000.0
Saving figure spill for GA
1000.0
Saving figure spill for GP
1000.0
Saving figure spill for OH


In [19]:
plt.close('all')

In [20]:
#defining storage plot function
def plot_storage(a,b,shifted,scale=scale,n=n_stages):
    if scale:
        constant=1e6
    else:
        constant=1
    print(constant)
    q_0=a.iloc[0][1:].to_numpy()*constant
    q_10=a.iloc[1][1:].to_numpy()*constant
    q_25=a.iloc[2][1:].to_numpy()*constant
    q_50=a.iloc[3][1:].to_numpy()*constant
    q_75=a.iloc[4][1:].to_numpy()*constant
    q_90=a.iloc[5][1:].to_numpy()*constant
    q_100=a.iloc[6][1:].to_numpy()*constant
    top=np.full(shape=n,fill_value=b["baseline"],)
    base=np.full(shape=n,fill_value=b["flood_control"],)
    x=np.arange(0,n)
    x=x+1
    xx=np.arange(0,len(shifted))
    xx=xx+1
    fig=plt.figure(figsize=(40,20),dpi=300)
    ax=fig.add_subplot(111,label='1')
    c='#33d7ff'
    ax.plot(x,q_50,color='#3337ff',linewidth=5,label='Median')
    ax.plot(x,top,color='k',linewidth=5,label='Top Exclusive Flood Control')
    ax.plot(x,base,color='r',linewidth=5,label='Base Exclusive Flood Control')
    ax.plot(xx,shifted*constant,'g:',linewidth=5,label="roling horizon")
    ax.fill_between(x,q_0,q_10,alpha=.1,color=c)
    ax.fill_between(x,q_10,q_25,alpha=.3,color=c)
    ax.fill_between(x,q_25,q_50,alpha=.5,color=c)
    ax.fill_between(x,q_50,q_75,alpha=.3,color=c)
    ax.fill_between(x,q_75,q_90,alpha=.2,color=c)
    ax.fill_between(x,q_90,q_100,alpha=.1,color=c)
    ax.fill_between(x,top,base,alpha=.3,color='r')
    
    title=b['name']
    plt.title(title,fontsize=30)
    plt.xlabel(f'Stage ({n} day horizon)',fontsize=30)
    plt.ylabel('Storage (AF)',fontsize=30)
    plt.grid(True,which='major',axis='both')
    plt.xticks(ticks=x)
    ax.tick_params(axis='both', which='major', labelsize=30)
    ax.yaxis.get_offset_text().set_size(30)
    plt.xlim([1,n])
    ax.legend(fontsize=30)
    save_fig(f'storage for {title}')
    plt.close('all')

In [21]:
#plotting for storage
kk=0
for dam,i in zip(storage,range(len(name))):
    p=params.iloc[i,:]
    plot_storage(dam,p,shifted_solutions.iloc[16:17+7,kk])
    kk+=1

1000000.0
Saving figure storage for BB
1000000.0
Saving figure storage for FP
1000000.0
Saving figure storage for FR
1000000.0
Saving figure storage for GA
1000000.0
Saving figure storage for GP
1000000.0
Saving figure storage for OH


In [22]:
plt.close('all')

In [23]:
print('Plotting is Done!')

Plotting is Done!
